In [1]:
import pandas as pd
import numpy as np

## Prepare purchases

In [2]:
purchases = pd.read_csv('../dataframes/retail_train.csv')

In [3]:
purchases = purchases[['user_id', 'item_id', 'quantity', 'sales_value']]
purchases.head(2)

,user_id,item_id,quantity,sales_value
0,2375,1004906,1,1.39
1,2375,1033142,1,0.82


In [4]:
purchases.count()

user_id        2396804
item_id        2396804
quantity       2396804
sales_value    2396804
dtype: int64

In [5]:
purchases = purchases[(purchases.sales_value > 0) & (purchases.quantity > 0)]
purchases.user_id.count()

2379348

In [6]:
purchases.to_csv('./data/purchases.csv', index=False)

## Get top 5000 for reduce item_features dataframe

In [7]:
top_5000 = purchases[purchases.sales_value / purchases.quantity >= 1] \
    .groupby('item_id')['quantity'].sum() \
    .reset_index() \
    .sort_values('quantity', ascending=False) \
    .head(5000) \
    .item_id.tolist()

## Prepare product features

In [8]:
item_features = pd.read_csv('../dataframes/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.head(2)

,product_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [9]:
item_features = item_features[['product_id', 'department', 'commodity_desc']]
item_features.rename(columns={'commodity_desc': 'category'}, inplace=True)
item_features.head(2)

,product_id,department,category
0,25671,GROCERY,FRZN ICE
1,26081,MISC. TRANS.,NO COMMODITY DESCRIPTION


In [10]:
total_rows = item_features.product_id.count()
unique_ids = item_features.product_id.nunique()

print(f'total_rows: {total_rows}\nunique_ids: {unique_ids}')

total_rows: 92353
unique_ids: 92353


In [11]:
item_features = item_features[item_features['product_id'].isin(top_5000)]

item_features

,product_id,department,category
5809,818981,GROCERY,COLD CEREAL
5816,819063,GROCERY,PASTA SAUCE
5819,819112,GROCERY,CANNED JUICES
5830,819255,MEAT-PCKGD,BREAKFAST SAUSAGE/SANDWICHES
5836,819304,GROCERY,PASTA SAUCE
...,...,...,...
89273,15926887,GROCERY,BAG SNACKS
89281,15926927,GROCERY,BAG SNACKS
89618,15972074,GROCERY,BATH TISSUES
89664,15972298,GROCERY,BATH TISSUES


In [12]:
item_features.department.nunique()

21

In [13]:
item_features.category.nunique()

202

In [14]:
item_features.to_csv('./data/item_features.csv', index=False)

In [15]:
item_features.count()

product_id    5000
department    5000
category      5000
dtype: int64

## Top 10 popular items from different category

In [16]:
item_features[item_features['product_id'].isin(top_5000[:10])]

,product_id,department,category
16015,908531,GROCERY,FLUID MILK PRODUCTS
20973,951590,GROCERY,BAKED BREAD/BUNS/ROLLS
24250,981760,GROCERY,EGGS
25754,995242,GROCERY,FLUID MILK PRODUCTS
29657,1029743,GROCERY,FLUID MILK PRODUCTS
35576,1082185,PRODUCE,TROPICAL FRUIT
38262,1106523,GROCERY,FLUID MILK PRODUCTS
40600,1127831,PRODUCE,BERRIES
41175,1133018,GROCERY,FLUID MILK PRODUCTS
53097,5569230,GROCERY,SOFT DRINKS


In [17]:
i = 0
category_list = []
top_5 = []

for id in top_5000: 
    if (i == 5):
        break
    category = item_features.loc[item_features['product_id'] == id, 'category'].values[0]
    if (not category in category_list):
        category_list.append(category)
        top_5.append(id)
        i = i + 1


In [20]:
top_5

[995242, 1082185, 1127831, 5569230, 951590]

In [28]:
top_5_df = pd.DataFrame(top_5, columns=['product_id'])
top_5_df.head()

,product_id
0,995242
1,1082185
2,1127831
3,5569230
4,951590


In [29]:
top_5_df.to_csv('./data/top_5_by_category.csv', index=False)